# Langchain Tutorial

#### 1. Basic Setup

In [ ]:
# Configure langsmith for evaluation
# LANGSMITH_TRACING=True
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_API_KEY=""
# LANGSMITH_PROJECT="langchain_tutorial"

In [ ]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")  # Use the correct API version

In [27]:
# Setup azure openai connection
model = AzureChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ["AZURE_OPENAI_API_VERSION"])

In [ ]:
# Process invoke
result = model.invoke("What is 81 dividied by 9?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)

#### 2. Chat Model Basic Conversation

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
# System message always comes first

messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?")
]

result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

In [ ]:
# Continuing the message chain
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content=result.content),
    HumanMessage(content="What is 50 divided by 5?")
]

result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

#### 4. Chat Model Conversation with User

(Skipped 3 which focused on how to defined other model apis and work with them on langchain)

Aim is to have real-time conversations similar to chatgpt website

In [5]:
# importing from langchain schema and not langchain messages as was earlier
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [6]:
chat_history = [] #Use a list to store messages

system_message = SystemMessage(content="You are a helpful AI assistant")
chat_history.append(system_message) # add system messages to chat history

In [9]:
# Create the chat loop for continued conversations

while True:
    query = input("You: ")
    # conversation stop condition when user says exit
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query)) # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response)) # Add AI message

    print (f"AI: {response}")

print("----Message History-------")
print(chat_history)


AI: As of my last update in October 2023, there is no widely recognized entity or term "langsmith" that stands out in prominent categories such as technology, history, culture, or popular media. It's possible that it could refer to something niche, a new development, or a term not widely circulated or recognized in major databases or sources.

If "langsmith" is a new technology, company, concept, or a specific term, I may not have information about it. Could you provide more context or specify the area where you've encountered this term? This might help me give you more accurate information.
AI: As of my last update in October 2023, I don't have specific information about "LangFlow." It’s possible that it could be a new product, service, company, term, or concept that has emerged recently or isn’t widely recognized in major available sources. 

If it’s related to a specific field (such as technology, business, art, etc.), could you provide more context? That way, I might be able to off

#### 5. Save Message History in the Cloud - Google Firestore

**(WILL RETURN TO THIS LATER. IT IS USED TO SAVE CHAT SESSIONS IN CASE WE WANT TO CALL IT IN THE FUTURE AND NOT START OVER.)**

Steps to replicate this example:
1. Create a Firebase account
2. Create a new Firebase project
    - Copy the project ID
3. Create a Firestore database in the Firebase project
4. Install the Google Cloud CLI on your computer
    - https://cloud.google.com/sdk/docs/install
    - Authenticate the Google Cloud CLI with your Google account
        - https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev
    - Set your default project to the new Firebase project you created
5. Enable the Firestore API in the Google Cloud Console:
    - https://console.cloud.google.com/apis/enableflow?apiid=firestore.googleapis.com&project=crewai-automation


# 6. Prompt Template

Nice way to structure prompts between system and user input

In [29]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [33]:
# Part 1: Create a ChatPromptTemplate using a template string
template  = "Tell me a joke about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)

In [31]:
print("----Prompt from Template-----")
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

----Prompt from Template-----
messages=[HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]


In [ ]:
# PART 3: Prompt with System and Human Messages (Using Tuples)
# Failing to put it in tuples = no replacement of eleemnts in the curly brackets
messages = [
    ("system", "You are a comedian who tells jokes about {topic}"),
    ("human", "Tell me a {joke_count} jokes."),
]

In [35]:
prompt_template = ChatPromptTemplate.from_messages(messages)


In [36]:
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print("\n---- Prompt with System and Human Messages (Tuple)----\n")
print(prompt)


---- Prompt with System and Human Messages (Tuple)----

messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a 3 jokes.', additional_kwargs={}, response_metadata={})]


# 7. Prompt Template with Chat Model

In [37]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [39]:
template  = "Tell me a joke about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)
prompt = prompt_template.invoke({"topic": "cats"})
result = model.invoke(prompt)
print(result.content)

Why don't cats play poker in the jungle?

Too many cheetahs!


In [40]:
# Prompt template with system and human message (tuples)
messages = [
    ("system", "You are a comedian who tells jokes about {topic}"),
    ("human", "Tell me a {joke_count} jokes."),
]

In [41]:
prompt_template = ChatPromptTemplate.from_messages(messages)


In [42]:
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
result = model.invoke(prompt)
print(result.content)

Absolutely, here are three lawyer jokes for you:

1. Why don’t sharks attack lawyers? 
   Professional courtesy.

2. How can you tell if a lawyer is lying?
   Other lawyers look interested.

3. What's the difference between a lawyer and a herd of buffalo?
   The lawyer charges more. 

Hope these bring a smile to your face!


# 8. Chains

Tying together a series of task using LCEL (lang chain expression language)

Chains can be dag structures like fork, collider and chain. Branching structure is possible representing different paths based on outputs of previous steps.

In [43]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [ ]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")  # Use the correct API version

In [45]:
# Setup azure openai connection
model = AzureChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ["AZURE_OPENAI_API_VERSION"])

In [48]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}"),
        ("human", "Tell me a {joke_count} jokes"),
    ]
)

In [50]:
# Create the combined chain using Langchain expression language (LCEL)
chain = prompt_template | model | StrOutputParser()

In [52]:
result = chain.invoke({"topic": "lawyers", "joke_count":3 })

In [53]:
print(result)

Sure, here are three lawyer jokes for you:

1. Why don’t sharks attack lawyers?
   Because of professional courtesy.

2. What's the difference between a good lawyer and a bad lawyer?
   A bad lawyer might let a case drag on for several years. A good lawyer knows how to make it last even longer.

3. How many lawyer jokes are there, anyway?
   Just three. The rest are true stories.


# 9. Chains Under-The-Hood

In [54]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence

In [55]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}"),
        ("human", "Tell me a {joke_count} jokes"),
    ]
)

In [56]:
# Create individual runnable (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

In [57]:
# Create runnable sequence equivalent to the LCEL chain
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

In [58]:
response = chain.invoke({"topic": "lawyers", "joke_count": 3})
print(response)

Sure, here are three jokes about lawyers for you:

1. **Why don't sharks attack lawyers?**
   Because of professional courtesy.

2. **What’s the difference between a lawyer and a herd of buffalo?**
   The lawyer charges more.

3. **Why did the lawyer show up in court in his underwear?**
   He wanted to make a brief appearance.

Hope these brought a smile to your face!


# 10. Chains Extended

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser
from langchain.schema.runnable import RunnableLambda

In [61]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}"),
        ("human", "Tell me a {joke_count} jokes"),
    ]
)

In [62]:
# Define additional processing steps using RunnabelLambda
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

In [64]:
chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

In [65]:
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

In [66]:
print(result)

Word count: 47
SURE, HERE ARE THREE LAWYER JOKES FOR YOU:

1. WHY DON'T SHARKS ATTACK LAWYERS?
   BECAUSE OF PROFESSIONAL COURTESY!

2. WHAT'S THE DIFFERENCE BETWEEN A LAWYER AND A HERD OF BUFFALO?
   THE LAWYER CHARGES MORE.

3. WHAT'S BLACK AND BROWN AND LOOKS GOOD ON A LAWYER?
   A DOBERMAN.


# 11. Parallel Chains

First define the functions properly and then introduce lambda functions before putting it inside the chain

In [67]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda

In [68]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)

In [69]:
# Define pros
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


In [75]:
# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer. You are also bief with your response"),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)

In [76]:
# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


In [77]:
# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)


In [78]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)


In [79]:
# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})
print(result)


Pros:
The latest MacBook Pro models come with numerous features that collectively contribute to their reputation as high-performance, reliable laptops. Here's a breakdown of the pros for each of the main features:

1. **M1 Pro and M1 Max Chips**:
   - **Pros**: Significantly better performance and energy efficiency compared to Intel-based models, allowing for faster processing, seamless multitasking, and longer battery life.

2. **Display**:
   - **Liquid Retina XDR Display**: 
     - **Pros**: Exceptionally high brightness, deep blacks, and an extensive P3 color gamut result in stunning visuals, making it ideal for creative professionals involved in photo and video editing.
   - **ProMotion Technology**:
     - **Pros**: Up to 120Hz refresh rate provides smoother scrolling and more responsive display, enhancing the overall user experience.

3. **Design**:
   - **Aluminum Unibody**:
     - **Pros**: Sleek, durable, and premium feel, available in aesthetically pleasing colors (Space Gra

# 12. Branching

In [80]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

In [81]:
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

In [82]:
negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

In [83]:
neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)


In [84]:
escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)


In [85]:
classification_template = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}")
    ]
)

In [ ]:
# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x, positive_feedback_template | model | StrOutputParser() # positive feedback chain
    ),
    (
        lambda x: "negative" in x, negative_feedback_template | model | StrOutputParser() # negative feedback
    ),
    (
        lambda x: "neutral" in x, neutral_feedback_template | model | StrOutputParser() # netrual feedback
    ),
    escalate_feedback_template | model | StrOutputParser() # If classification fails then we need to set a default branch outside. Here escalate is the default branch.
)

In [87]:
# Create classification chain
classification_chain = classification_template | model | StrOutputParser()

In [88]:
# Combine classification and response generation into one chain
chain = classification_chain | branches

In [89]:
# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

# Output the result
print(result)

Subject: Urgent Attention Required: Negative Feedback Escalation

Dear [Human Agent's Name],

I hope this message finds you well. 

We have received a piece of negative feedback from a customer that requires immediate attention and resolution. Due to the nature of the feedback, it is important that we address the issue promptly to ensure customer satisfaction and to maintain our service standards.

The details of the feedback are as follows:
- Feedback Type: Negative
- Customer Name: [Customer Name]
- Date Received: [Date]
- Specific Concerns: [Provide a brief summary of the feedback, highlighting key issues or complaints]

Given the impact this feedback could have on our reputation and customer relationships, I kindly request that you prioritize this case and take necessary actions to resolve the concerns raised by the customer. Please reach out to the customer directly to understand their perspective and offer suitable solutions or compensations as needed.

Feel free to loop in any r